In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
PATH = "data/mnist/"
sz=224

In [4]:
#fileid save imageid
import os
filelist = os.listdir(f'{PATH}/test')
fileid = [filename.split('.')[0] for filename in filelist]

##  unfreeze and training model

In [8]:
!rm -rf {PATH}tmp

In [5]:
arch=resnet34
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms,test_name='test')
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [7]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])

In [11]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=6)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch:   0, train_loss: 0.095370, val_loss: 0.055628, accuracy: 0.981934
epoch:   1, train_loss: 0.064242, val_loss: 0.031375, accuracy: 0.990234
epoch:   2, train_loss: 0.050192, val_loss: 0.027299, accuracy: 0.989746
epoch:   3, train_loss: 0.047826, val_loss: 0.017025, accuracy: 0.993164
epoch:   4, train_loss: 0.026087, val_loss: 0.014423, accuracy: 0.995605
epoch:   5, train_loss: 0.024882, val_loss: 0.008110, accuracy: 0.998047
epoch:   6, train_loss: 0.022084, val_loss: 0.008820, accuracy: 0.997070



In [50]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch:   0, train_loss: 0.007702, val_loss: 0.000731, accuracy: 1.000000
epoch:   1, train_loss: 0.008071, val_loss: 0.001146, accuracy: 0.999512
epoch:   2, train_loss: 0.004463, val_loss: 0.001583, accuracy: 0.999023



In [60]:
learn.save('ResNet34_all_3x8')  #cycle_mult6+2 &TTA 0.99628

In [8]:
learn.load('ResNet34_all_3x8') 

In [9]:
learn.fit(lr, 1, cycle_len=1, cycle_mult=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  3%|▎         | 21/625 [00:08<03:56,  2.55it/s, loss=0.00633]


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



epoch      trn_loss   val_loss   accuracy                      
    0      0.012558   0.0007     1.0       



[0.00070012314, 1.0]

## output the result

In [53]:
test_log_preds = learn.predict(is_test=True)
test_probs = np.exp(test_log_preds[:,1])
test_preds = np.argmax(test_log_preds, axis=1)
subm = np.stack([fileid,test_preds], axis=1)

In [10]:
#TTA
test_log_preds_tta,y = learn.TTA(is_test=True)
mean_test_log_preds_tta = np.mean(test_log_preds_tta, axis=0)
test_preds_tta = np.argmax(mean_test_log_preds_tta, axis=1)
subm = np.stack([fileid,test_preds_tta], axis=1)

In [11]:
%cd $PATH
submission_file_name = 'submission_v13.01.csv'
np.savetxt(submission_file_name, subm, fmt='%s,%s', header='ImageId,Label', comments='')

/home/paperspace/data/mnist


In [55]:
from IPython.display import FileLink
%cd ../..
FileLink('data/mnist/' + submission_file_name)

/home/paperspace


/home/paperspace/data/mnist/submission_v13.01.csv

## 预测结果问题分析

In [ ]:
!rm -rf {PATH}tmp

In [ ]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz),test_name='test')
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 1)

In [ ]:
test_log_preds = learn.predict(is_test=True)
test_probs = np.exp(test_log_preds[:,1])
test_preds = np.argmax(test_log_preds, axis=1)

In [ ]:
def plot_test_with_title(idxs):
    imgs = np.stack([data.test_ds[x][0] for x in idxs])
    return plots(data.test_ds.denorm(imgs), rows=1)

In [ ]:
test_preds[:10]

In [ ]:
idxs = np.arange(0,10)
plot_test_with_title(idxs)